# <center> 🧽 Data Cleaning Part 1 </center>

## Table of Contents

- [1. Introduction](#1-introduction)
- [2. Loading Data](#2-loading-data)

## 1. Introduction
[Back to Table of Contents](#table-of-contents)

---

| ⚡ Description: Introducing the project ⚡ |
| :--------------------------- |

---

<a id="two"></a>
## 2. Loading Data
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data in our environment ⚡ |
| :--------------------------- |

---

### Import packages

In [1]:
# Import packages
import re

import numpy as np
import pandas as pd

### Rental Properties

In [2]:
# Load the rental listings dataset
rental_listings = pd.read_csv("../data/raw/rental_listings_for_capital_cities.csv")

# Display the first few rows of the dataset and it's shape
display("Rentals", rental_listings.head(), rental_listings.shape)

'Rentals'

,price,location,property_title,property_description,property_features,listing_slug
0,R 7 000Per Month,"Bhisho, Tyu Tyu North",3 Bedroom House to Rent in Tyu Tyu North,RE/MAX Border is proud to present this 3-bedro...,Property OverviewListing Number115295115Type o...,3-bedroom-house-to-rent-in-tyu-tyu-north-11529...
1,R 7 000Per Month,"Bhisho, Bhisho Central",3 Bedroom House to Rent in Bhisho Central,How CONVENIENT is this flat directly opposite ...,Property OverviewListing Number115332086Type o...,3-bedroom-house-to-rent-in-bhisho-central-1153...
2,R 5 300Per Month,"Bhisho, Tyu Tyu North",2 Bedroom House to Rent in Tyu Tyu North,Comfortable Home for a family which is availab...,Property OverviewListing Number115210602Type o...,2-bedroom-house-to-rent-in-tyu-tyu-north-11521...
3,NaN,NaN,NaN,NaN,NaN,house-to-rent-in-bisho-park-101731048
4,R 7 500Per Month,"Bhisho, Bisho Park",3 Bedroom House to Rent in Bisho Park,BetterProp is thrilled to present this well-ma...,Property OverviewListing Number115555490Type o...,3-bedroom-house-to-rent-in-bisho-park-115555490


(15324, 6)

In [3]:
# Print a summary of the dataset
rental_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15324 entries, 0 to 15323
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   price                 15323 non-null  object
 1   location              8189 non-null   object
 2   property_title        15323 non-null  object
 3   property_description  15323 non-null  object
 4   property_features     15323 non-null  object
 5   listing_slug          15324 non-null  object
dtypes: object(6)
memory usage: 718.4+ KB


In [4]:
# Print the total number of null values in the dataset
rental_null_values = rental_listings.isnull().sum().sum()
print(f"There are {rental_null_values} null values in the rental dataset")

There are 7139 null values in the rental dataset


In [5]:
# Print the number of null values in each column
rental_listings.isnull().sum()

price                      1
location                7135
property_title             1
property_description       1
property_features          1
listing_slug               0
dtype: int64

In [6]:
rental_listings[rental_listings.isnull().sum(axis=1) >= 4]

,price,location,property_title,property_description,property_features,listing_slug
3,NaN,NaN,NaN,NaN,NaN,house-to-rent-in-bisho-park-101731048


In [7]:
rental_listings.drop(
    rental_listings[rental_listings.isnull().sum(axis=1) >= 2].index, inplace=True)

In [8]:
# Get the total number of duplicate rows in the dataset
rental_duplicates = rental_listings[rental_listings.duplicated()]
num_rental_duplicates = len(rental_duplicates)
print(f"There are {num_rental_duplicates} duplicates in the rental dataset ")

There are 1303 duplicates in the rental dataset 


In [9]:
# Drop duplicate rows and keep the first occurrence
rental_listings = rental_listings.drop_duplicates().reset_index(drop=True)

#### Property features extraction

In [10]:
# Remove any information following the text 'Points of Interest' in the 'property_features' column
rental_listings["property_features"] = rental_listings["property_features"].str.replace(
    r'Points of Interest.*', '', regex=True
)

rental_listings["property_features"] = rental_listings["property_features"].str.strip()

In [11]:
patterns = {
    'Listing Number': r'Listing Number(\d+)',
    'Type of Property': r'Type of Property\s*(House|Apartment / Flat|Townhouse|Commercial|Industrial|Vacant Land|Farm)',
    'Pets Allowed': r'Pets Allowed(Yes|No)',
    'Furnished': r'Furnished(Yes|No)',
    'Bedrooms': r'Bedrooms?\s*(\d+)',
    'Bathrooms': r'Bathrooms?\s*(\d+)',
    'Kitchen': r'Kitchens?\s*(\d+)',
    'Lounge': r'Lounges?\s*(\d+)',
    'Dining Room': r'Dining Room(\d+)',
    'Parking': r'Parking(\d+)',
    'Backup Water': r'Backup Water(Yes|No)',
    'Erf Size (m²)': r'Erf Size\s*(\d+)\s*m²',
    'Floor Area (m²)': r'Floor Area\s*(\d+)\s*m²',
    'Garden': r'Gardens?\s*(\d+)',
    'Pool': r'Pools?\s*(\d+)',
    'Garage': r'Garages?\s*(\d+)'
}



# Function to extract features from text
def extract_features(text):
    return {
        field: (match := re.search(pattern, text)
                ) and match.group(1).strip() or None for field, pattern in patterns.items()
    }
    
# Apply extraction row-wise
extracted_df = rental_listings['property_features'].apply(extract_features).apply(pd.Series)

# Combine original df with the extracted columns
results_df = pd.concat([rental_listings, extracted_df], axis=1)

# View result
results_df.head()

,price,location,property_title,property_description,property_features,listing_slug,Listing Number,Type of Property,Pets Allowed,Furnished,...,Kitchen,Lounge,Dining Room,Parking,Backup Water,Erf Size (m²),Floor Area (m²),Garden,Pool,Garage
0,R 7 000Per Month,"Bhisho, Tyu Tyu North",3 Bedroom House to Rent in Tyu Tyu North,RE/MAX Border is proud to present this 3-bedro...,Property OverviewListing Number115295115Type o...,3-bedroom-house-to-rent-in-tyu-tyu-north-11529...,115295115,House,No,No,...,1,1,1,1,No,None,None,None,None,None
1,R 7 000Per Month,"Bhisho, Bhisho Central",3 Bedroom House to Rent in Bhisho Central,How CONVENIENT is this flat directly opposite ...,Property OverviewListing Number115332086Type o...,3-bedroom-house-to-rent-in-bhisho-central-1153...,115332086,House,No,No,...,1,1,None,1,None,None,None,1,None,None
2,R 5 300Per Month,"Bhisho, Tyu Tyu North",2 Bedroom House to Rent in Tyu Tyu North,Comfortable Home for a family which is availab...,Property OverviewListing Number115210602Type o...,2-bedroom-house-to-rent-in-tyu-tyu-north-11521...,115210602,House,Yes,No,...,1,1,None,1,None,None,None,1,None,None
3,R 7 500Per Month,"Bhisho, Bisho Park",3 Bedroom House to Rent in Bisho Park,BetterProp is thrilled to present this well-ma...,Property OverviewListing Number115555490Type o...,3-bedroom-house-to-rent-in-bisho-park-115555490,115555490,House,Yes,No,...,1,None,None,3,None,None,None,None,None,None
4,R 5 500Per Month,"Bhisho, Bisho Park",House to Rent in Bisho Park,This neat family home boasts:3 bedroomsOpen pl...,Property OverviewListing Number115525482Type o...,house-to-rent-in-bisho-park-115525482,115525482,House,None,None,...,None,None,None,None,None,None,None,None,None,None


In [12]:
results_df["Bedrooms"].isnull().sum()

np.int64(11068)

In [13]:
def extract_bedrooms_from_property_title(title):
    if pd.isna(title):
        return None
    match = re.search(r'(\d+)[-\s]?bedroom', title.lower())
    return int(match.group(1)) if match else None

results_df['Bedrooms'] = results_df['Bedrooms'].fillna(
    results_df.apply(lambda row: extract_bedrooms_from_property_title(row['property_title']) if pd.isna(row['Bedrooms']) else row['Bedrooms'], axis=1))

# Optional: convert to integer if needed
results_df['Bedrooms'] = pd.to_numeric(results_df['Bedrooms'], errors='coerce').astype('Int64')

In [14]:
results_df["Bedrooms"].isnull().sum()

np.int64(8521)

####  Suburb, city and province extraction

In [15]:
# Extract the suburb from the property_title column
results_df['suburb'] = results_df['property_title'].str.extract(
    r'to Rent in (.+)', expand=False)

# Remove the trailing numbers from the suburb column
results_df['suburb'] = (
    results_df['suburb'].str.replace(r'-\d+$', '', regex=True).str.strip())

# Drop the property_title column as we have exytacted all the information we need
results_df.drop(columns=['property_title'], inplace=True)

# Print the first five rows of the table to see the results
results_df.head()

,price,location,property_description,property_features,listing_slug,Listing Number,Type of Property,Pets Allowed,Furnished,Bedrooms,...,Lounge,Dining Room,Parking,Backup Water,Erf Size (m²),Floor Area (m²),Garden,Pool,Garage,suburb
0,R 7 000Per Month,"Bhisho, Tyu Tyu North",RE/MAX Border is proud to present this 3-bedro...,Property OverviewListing Number115295115Type o...,3-bedroom-house-to-rent-in-tyu-tyu-north-11529...,115295115,House,No,No,3,...,1,1,1,No,None,None,None,None,None,Tyu Tyu North
1,R 7 000Per Month,"Bhisho, Bhisho Central",How CONVENIENT is this flat directly opposite ...,Property OverviewListing Number115332086Type o...,3-bedroom-house-to-rent-in-bhisho-central-1153...,115332086,House,No,No,3,...,1,None,1,None,None,None,1,None,None,Bhisho Central
2,R 5 300Per Month,"Bhisho, Tyu Tyu North",Comfortable Home for a family which is availab...,Property OverviewListing Number115210602Type o...,2-bedroom-house-to-rent-in-tyu-tyu-north-11521...,115210602,House,Yes,No,2,...,1,None,1,None,None,None,1,None,None,Tyu Tyu North
3,R 7 500Per Month,"Bhisho, Bisho Park",BetterProp is thrilled to present this well-ma...,Property OverviewListing Number115555490Type o...,3-bedroom-house-to-rent-in-bisho-park-115555490,115555490,House,Yes,No,3,...,None,None,3,None,None,None,None,None,None,Bisho Park
4,R 5 500Per Month,"Bhisho, Bisho Park",This neat family home boasts:3 bedroomsOpen pl...,Property OverviewListing Number115525482Type o...,house-to-rent-in-bisho-park-115525482,115525482,House,None,None,<NA>,...,None,None,None,None,None,None,None,None,None,Bisho Park


In [16]:
# Load the csv that contains the capital_cities with their respective provinces and create a mapping
city_province = pd.read_csv("../data/raw/south_africa_provinces_capitals.csv")
city_list = city_province['city'].dropna().str.strip().unique()

In [17]:
def extract_city_from_location(location, city_list):
    if pd.isna(location):
        return None
    
    
    location_lower = location.lower()
    for city in city_list:
        if city.lower() in location_lower:
            return city
    return None

In [18]:
# Apply the extract_city_from_location function to extract the city from the location column
results_df['city'] = results_df['location'].apply(
    lambda loc: extract_city_from_location(loc, city_list))

# Drop the location column as it is redundant
results_df.drop(columns=["location"], inplace=True)

In [19]:
# Create a mapping of suburb to known cities
suburb_to_city_map = results_df[results_df['city'].notna()].drop_duplicates(subset=['suburb', 'city']).set_index('suburb')['city'].to_dict()

# Fill missing city values based on the suburb
results_df['city'] = results_df.apply(lambda row: suburb_to_city_map.get(row['suburb'], row['city']) if pd.isna(row['city']) else row['city'], axis= 1)   

In [20]:
results_df['city'].isnull().sum()

np.int64(91)

In [21]:
# Drop the city rows with null values
results_df = results_df.dropna(subset=['city'])

In [22]:
results_df['city'].isnull().sum()

np.int64(0)

In [23]:
# Merge the city with its matching province
results_df = results_df.merge(city_province, on='city', how='left')

In [24]:
# Drop the listing slug column
results_df.drop(columns=['listing_slug'], inplace=True)

#### Price column

In [25]:
# Extract the pricing option after the price amount
results_df['price_type'] = results_df['price'].str.extract(
    r'(Per\s*Month|Per\s*Day|m2|m²|Perm²)', flags=re.IGNORECASE, expand=False
)


# Normalize price_type text (remove spaces, lower case) then map to standard form
results_df['price_type'] = results_df['price_type'].str.replace(
    r'\s+', '', regex=True).str.lower()
results_df['price_type'] = results_df['price_type'].replace({
    'permonth': 'Per Month',
    'perday': 'Per Day',
    'm2': 'Per m²',
    'm²': 'Per m²',
    'perm²': 'Per m²'
})

# Remove spaces inside the price string first
results_df['price_clean'] = results_df['price'].str.replace(
    r'\s+', '', regex=True)

# Now extract the numeric part (digits only, continuous)
results_df['price (in rands)'] = results_df['price_clean'].str.extract(
    r'R?(\d+)', expand=False)

# Convert the price(in rands) column to numeric
results_df['price (in rands)'] = pd.to_numeric(
    results_df['price (in rands)'], errors='coerce')

# Drop the helper clean column if you want
results_df.drop(columns=['price_clean'], inplace=True)

# Then clean blank or whitespace-only price_type entries as before
results_df['price_type'] = results_df['price_type'].replace(
    r'^\s*$', pd.NA, regex=True)

# Preview results
print(results_df[['price', 'price_type', 'price (in rands)']].head())

              price price_type  price (in rands)
0  R 7 000Per Month  Per Month            7000.0
1  R 7 000Per Month  Per Month            7000.0
2  R 5 300Per Month  Per Month            5300.0
3  R 7 500Per Month  Per Month            7500.0
4  R 5 500Per Month  Per Month            5500.0


In [26]:
# Get the unique values in the price_type column
results_df['price_type'].value_counts()

price_type
Per Month    12826
Per m²         948
Per Day        143
Name: count, dtype: int64

In [27]:
# Get the number of null values in the price_type column
results_df['price_type'].isnull().sum()

np.int64(12)

In [28]:
# Visualize the rows where price_type is null
results_df[results_df['price_type'].isna()]

,price,property_description,property_features,Listing Number,Type of Property,Pets Allowed,Furnished,Bedrooms,Bathrooms,Kitchen,...,Erf Size (m²),Floor Area (m²),Garden,Pool,Garage,suburb,city,province,price_type,price (in rands)
357,POA,"Unit 1 within 29 Grey Street, Hilton - FS, Blo...",Property OverviewListing Number114774738Type o...,114774738,Industrial,None,No,<NA>,None,None,...,None,None,None,None,None,Hilton,Bloemfontein,Free State,NaN,NaN
3728,POA,"Salty ocean breezes, golden sunsets, breathtak...",Property OverviewListing Number115605901Type o...,115605901,House,No,Yes,4,6,2,...,None,None,1,1,1,Bantry Bay,Cape Town,Western Cape,NaN,NaN
3760,POA,Peering over the edge of Africa from its cliff...,Property OverviewListing Number115594664Type o...,115594664,House,No,Yes,5,7,2,...,None,None,None,1,1,Clifton,Cape Town,Western Cape,NaN,NaN
3771,POA,Drawing inspiration from the whitewashed villa...,Property OverviewListing Number115613951Type o...,115613951,House,None,Yes,5,5,2,...,None,None,None,1,1,Fresnaye,Cape Town,Western Cape,NaN,NaN
3837,POA,"With its gaze fixed to the horizon, this beach...",Property OverviewListing Number115606187Type o...,115606187,House,No,Yes,4,5,1,...,None,None,None,1,2,Clifton,Cape Town,Western Cape,NaN,NaN
4180,POA,An exceptional and very glamorous Llandudno vi...,Property OverviewListing Number108778518Type o...,108778518,House,None,Yes,5,5,1,...,991,862,None,None,None,Llandudno,Cape Town,Western Cape,NaN,NaN
4587,POA,*Please note that the daily rate ranges from R...,Property OverviewListing Number115118688Type o...,115118688,House,No,Yes,5,4,1,...,530,None,None,None,None,Camps Bay,Cape Town,Western Cape,NaN,NaN
4610,POA,Clifton beach bungalow with entertainment area...,Property OverviewListing Number115153896Type o...,115153896,House,No,Yes,5,5,None,...,None,463,None,None,None,Clifton,Cape Town,Western Cape,NaN,NaN
4896,POA,A beautifully crafted furnished home with the ...,Property OverviewListing Number110813037Type o...,110813037,House,None,Yes,8,8,1,...,None,605,None,None,None,Llandudno,Cape Town,Western Cape,NaN,NaN
4899,POA,"Welcome to this Constantia homestead, steeped ...",Property OverviewListing Number113953599Type o...,113953599,House,None,Yes,7,7,None,...,None,None,None,None,None,Constantia,Cape Town,Western Cape,NaN,NaN


Identify price columns that are equal to POA and contain the price type and those without. 

In [29]:
# Get the number of rows in the dataset in the 'price' column that are equal to 'POA'
poa_mask = results_df['price'].astype(
    str).str.contains("POA", case=False, na=False)
num_poa = poa_mask.sum()

# Print the total number of rows
print(f"There are {num_poa} listings with 'POA' in the price column.")

There are 217 listings with 'POA' in the price column.


In [30]:
# Identify POA rows with valid price_type
poa_mask = results_df['price'].str.lower().str.startswith('poa')
poa_with_type_mask = poa_mask & results_df['price_type'].notna() & (results_df['price_type'] != '')

# Calculate averages for the suburb, Type of Property and price_type
avg_prices = results_df[~poa_mask & results_df['price (in rands)'].notna()] \
    .groupby(['suburb', 'Type of Property', 'price_type'])['price (in rands)'] \
    .apply(lambda x: pd.to_numeric(x, errors='coerce').mean()) \
    .reset_index().rename(columns={'price (in rands)': 'price_avg'})
    
# Merge the averages into the original DataFrame
results_df = results_df.merge(avg_prices, on=['suburb', 'Type of Property', 'price_type'], how='left')

# Replace the POA with non-empty price_types with the averages
results_df.loc[poa_with_type_mask,
'price (in rands)'] = results_df.loc[poa_with_type_mask, 'price_avg']

# Drop the POA rows without a price type
results_df = results_df[~(poa_mask & ~poa_with_type_mask)]

# Drop the helper column
results_df.drop(columns=['price_avg'], inplace=True)

In [31]:
print("Remaining 'POA':", results_df['price'].str.lower().eq('poa').sum())
print("Remaining nulls in 'price (in rands)':",
      results_df['price (in rands)'].isna().sum())

Remaining 'POA': 0
Remaining nulls in 'price (in rands)': 12


In [32]:
# Get the rows where price (in rands) is null
results_df[results_df['price (in rands)'].isna()]

,price,property_description,property_features,Listing Number,Type of Property,Pets Allowed,Furnished,Bedrooms,Bathrooms,Kitchen,...,Erf Size (m²),Floor Area (m²),Garden,Pool,Garage,suburb,city,province,price_type,price (in rands)
4246,POAPer Day,Positioned in the prestigious Fresnaye neighbo...,Property OverviewListing Number115374518Type o...,115374518,Apartment / Flat,None,None,4,4,1,...,None,None,None,1,None,Fresnaye,Cape Town,Western Cape,Per Day,NaN
4586,POAPer Day,Escape to Paradise...Welcome to a world of enc...,Property OverviewListing Number112682529Type o...,112682529,House,None,Yes,7,7,None,...,None,None,None,None,None,Zwaanswyk,Cape Town,Western Cape,Per Day,NaN
4836,POAPer Day,This magnificent penthouse located in Barley B...,Property OverviewListing Number111244618Type o...,111244618,Apartment / Flat,None,None,3,3,None,...,None,286,None,1,None,Bakoven,Cape Town,Western Cape,Per Day,NaN
4863,POAPer Day,Discover Luxury Living in the Heart of Cape To...,Property OverviewListing Number114968205Type o...,114968205,Apartment / Flat,None,None,4,4,1,...,None,None,None,1,None,Fresnaye,Cape Town,Western Cape,Per Day,NaN
4991,POAPer Day,*Available 01 May 2025 - 31 August 2025*If it ...,Property OverviewListing Number111859212Type o...,111859212,Apartment / Flat,None,Yes,3,4,1,...,None,None,None,1,None,Bantry Bay,Cape Town,Western Cape,Per Day,NaN
7450,POAPer m²,1.\tOffices with large windows2.\tKitchenette3...,Property OverviewListing Number115523149Type o...,115523149,Commercial,None,None,<NA>,None,None,...,None,235,None,None,None,Heriotdale,Johannesburg,Gauteng,Per m²,NaN
8317,POAPer m²,1.\tAmple Parking2.\tOffices with large window...,Property OverviewListing Number115418255Type o...,115418255,Commercial,None,None,<NA>,None,None,...,None,260,None,None,None,Heriotdale,Johannesburg,Gauteng,Per m²,NaN
8706,POAPer Month,Discover this spacious 500 square meter proper...,Property OverviewListing Number114459245Type o...,114459245,Industrial,None,No,<NA>,None,None,...,500,500,None,None,None,Westgate,Johannesburg,Gauteng,Per Month,NaN
8758,POAPer m²,1.\tOffices2.\tAmple Kitchenette3.\tAmple Ablu...,Property OverviewListing Number115517306Type o...,115517306,Commercial,None,None,<NA>,None,None,...,None,225,None,None,None,Heriotdale,Johannesburg,Gauteng,Per m²,NaN
12308,POAPer Month,"This rarely available 3,788sqm industrial ware...",Property OverviewListing Number113997397Type o...,113997397,Industrial,None,No,<NA>,None,None,...,None,None,None,None,None,Nelsville,Nelspruit,Mpumalanga,Per Month,NaN


In [33]:
# Drop rows in the price (in rands) column that are null
results_df = results_df.dropna(subset=['price (in rands)'])

In [34]:
# Drop the price column as we do not need it anymore
results_df.drop(columns=['price'], inplace=True)

### Pets Allowed

In [35]:
results_df['Pets Allowed'].isnull().sum()

np.int64(10304)

In [36]:
results_df['Pets Allowed'].value_counts()

Pets Allowed
No     2598
Yes    1003
Name: count, dtype: int64

In [37]:
# Calculate mode for each property type
pets_mode = results_df.groupby('Type of Property')['Pets Allowed'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA
)
print(pets_mode)

Type of Property
Apartment / Flat     No
Commercial           No
Farm                Yes
House               Yes
Industrial           No
Townhouse            No
Vacant Land          No
Name: Pets Allowed, dtype: object


In [38]:
default_pets_value = {
    'Apartment / Flat': 'No', 
    'Commercial': 'No',
    'Farm': 'Yes',
    'House': 'Yes', 
    'Industrial': 'No', 
    'Townhouse': 'No', 
    'Vacant Land': 'No'
}

results_df['Pets Allowed'] = results_df.apply(
    lambda row: default_pets_value[row['Type of Property']]
    if pd.isna(row['Pets Allowed']) and row['Type of Property'] in default_pets_value
    else row['Pets Allowed'],
    axis=1
)

In [39]:
print(results_df['Pets Allowed'].isna().sum())

0


### Furnished

In [40]:
results_df['Furnished'].isnull().sum()

np.int64(4635)

In [41]:
results_df['Furnished'].value_counts()

Furnished
No     7962
Yes    1308
Name: count, dtype: int64

In [42]:
# Calculate mode for each property type for the furnished column
furnished_mode = results_df.groupby('Type of Property')['Furnished'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA
)
print(furnished_mode)

Type of Property
Apartment / Flat    No
Commercial          No
Farm                No
House               No
Industrial          No
Townhouse           No
Vacant Land         No
Name: Furnished, dtype: object


In [43]:
# Fill the null vlaues in the furnished column with unknown
results_df['Furnished'] = results_df['Furnished'].fillna("Unknown")

In [44]:
print(results_df['Furnished'].isna().sum())

0


### Bedrooms

In [45]:
results_df['Bedrooms'].isnull().sum()

np.int64(8449)

In [46]:
results_df['Bedrooms'].value_counts()

Bedrooms
2     1825
1     1446
3     1178
5      403
4      347
0      177
6       45
7       17
8        8
9        4
11       2
13       2
12       1
10       1
Name: count, dtype: Int64

In [47]:
# Calculate mode for each property type for the furnished column
furnished_mode = results_df.groupby('Type of Property')['Bedrooms'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA
)
print(furnished_mode)

Type of Property
Apartment / Flat       2
Commercial             0
Farm                   2
House                  3
Industrial             2
Townhouse              2
Vacant Land         <NA>
Name: Bedrooms, dtype: Int64


In [48]:
bedroom_medians = results_df.groupby('Type of Property')['Bedrooms'].median()

def fill_bedrooms(row):
    if pd.isna(row['Bedrooms']):
        median = bedroom_medians.get(row['Type of Property'], np.nan)
        return median if pd.notna(median) else 0
    return row['Bedrooms']

results_df['Bedrooms'] = results_df.apply(fill_bedrooms, axis=1)

In [49]:
results_df['Bedrooms'].isnull().sum()

np.int64(0)

### Bathrooms

In [50]:
results_df['Bathrooms'].isnull().sum()

np.int64(6113)

In [51]:
results_df['Bathrooms'].value_counts()

Bathrooms
1      3937
2      2662
3       529
4       347
6        99
5        90
8        41
7        26
10       21
0        13
12        6
15        5
9         4
20        3
14        2
16        2
11        1
13        1
30        1
200       1
160       1
Name: count, dtype: int64

In [52]:
# Ensure 'Bathrooms' is numeric
results_df['Bathrooms'] = pd.to_numeric(
    results_df['Bathrooms'], errors='coerce')

# Filter listings with Bathrooms > 10
bathroom_outliers = results_df[results_df['Bathrooms'] > 9]

# Group by Type of Property and count
bathroom_summary = bathroom_outliers.groupby(
    'Type of Property')['Bathrooms'].count().reset_index()

# Rename column for clarity
bathroom_summary.columns = ['Type of Property', 'Count with >10 Bathrooms']

# Sort by count descending
bathroom_summary = bathroom_summary.sort_values(
    by='Count with >10 Bathrooms', ascending=False)

print(bathroom_summary)

  Type of Property  Count with >10 Bathrooms
0       Commercial                        21
2       Industrial                        19
1            House                         4


In [ ]:
threshold = 10
extreme_bathrooms = results_df[results_df['Bathrooms'] > threshold]s

   Type of Property  Median Bathrooms
2              Farm               3.0
1        Commercial               2.0
4        Industrial               2.0
3             House               2.0
5         Townhouse               2.0
0  Apartment / Flat               1.0
6       Vacant Land               NaN


In [54]:
# Calculate mode for each property type for the furnished column
furnished_mode = results_df.groupby('Type of Property')['Bathrooms'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else pd.NA
)
print(furnished_mode)

Type of Property
Apartment / Flat     1.0
Commercial           2.0
Farm                 1.0
House                1.0
Industrial           2.0
Townhouse            2.0
Vacant Land         <NA>
Name: Bathrooms, dtype: object


In [55]:
# Filter out rows where Bathrooms > 9
results_df = results_df[results_df['Bathrooms'] <= 9]

In [56]:
results_df['Bathrooms'].isnull().sum()

np.int64(0)

### Kitchen

In [57]:
results_df['Kitchen'].isnull().sum()

np.int64(1498)

In [58]:
results_df['Kitchen'] = pd.to_numeric(
    results_df['Kitchen'], errors='coerce')


# Group by property type and calculate median
median_bathrooms = results_df.groupby('Type of Property')[
    'Kitchen'].median().reset_index()

# Rename columns for clarity
median_bathrooms.columns = ['Type of Property', 'Median Kitchen']

# Sort by median descending (optional)
median_bathrooms = median_bathrooms.sort_values(
    by='Median Kitchen', ascending=False)

print(median_bathrooms)

   Type of Property  Median Kitchen
0  Apartment / Flat             1.0
1        Commercial             1.0
2              Farm             1.0
3             House             1.0
4        Industrial             1.0
5         Townhouse             1.0
